In [1]:
import numpy as np
import pandas as pd
from itertools import product

import gurobipy as grb

c:\Users\faust\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


Input Problem parameters

Définissons les variables

In [2]:
from extract_parameters import extract_parameters

################################
######## TO DO #################
################################

# choose taille among: "small", "medium", "large"
taille = "small"

In [3]:
################################
######## DO NOT CHANGE #########
################################
conge, qualif, d, p, b, besoin = extract_parameters(taille)

In [4]:
conge

[[1, 1, 0, 1, 1], [1, 0, 1, 1, 1], [1, 1, 1, 1, 1]]

Définissons les paramètres

In [5]:
from define_nums import define_nums

nombre_employes, horizon, nombre_qualif, nombre_projets = define_nums(conge, qualif, d, p, b, besoin)

In [6]:
## u_j, le projet j est réalisé
u = [0]*nombre_projets

## x_i_j_k_n, l'employé k travaille sur le projet j avec la qualif i
## au jour n
x = [[[[0]*horizon for _ in range(nombre_employes)] for _ in range(nombre_projets)] for _ in range(nombre_qualif)]

## begin_j, la date de debut effective du projet j
begin = [0]*nombre_projets

## end_j, la date de fin effective = date de livraison du projet j
end = [0]*nombre_projets

Model deployment

Add the variables

In [ ]:
model = grb.Model("Planning")

## Les dates de début des projets
dateDebut = model.addVar()

# Allocation of retailers to Division 1.
allocate = model.addVars(retailers, vtype=GRB.BINARY, name="allocate")

# Positive and negative deviation of delivery points goal.

deliveryPointsPos = model.addVar(ub= deliveryPoints5, name='deliveryPointsPos')
deliveryPointsNeg = model.addVar(ub= deliveryPoints5, name='deliveryPointsNeg')

Add constraints

In [ ]:
for i in range(nombre_qualif):
    for j in range(nombre_projets):
        for k in range(nombre_employes):
            for n in range(horizon):

                ## La qualification du personnel
                model.addConstr( x[i][j][k][n] <= qualif[i][k], name=f"QualificationPersonnel[{i}{k}]")

                ## Contraintes des congés
                model.addConstr( x[i][j][k][n] <= conge[k][n], name=f"CongesPersonnel[{k}{n}]")
        
        ## Contrainte d'unicité de réalisation d'un projet
        model.addConstr( grb.quickum(grb.quicksum(x[i][j][k][n] for n in range(horizon)) for k in range(nombre_employes)) <= besoin[i][j], name=f"UniciteRealisation[{i}{j}]")


## Unicité de l'affectation quotidienne
## WIP  !!!!!!!!!!!!
for k in range(nombre_employes):
    for n in range(horizon):
        y = [ [0]*nombre_projets for _ in range(nombre_qualif)]


        model.addConstr( grb.quickum(grb.quicksum(x[i][j][k][n] for j in range(nombre_projets)) for i in range(nombre_qualif)) <= 1, name=f"UniciteAffectation[{k}{n}]" )





## begin date
## TO DO


## end date
## TO DO

## Couverture des qualifications
CouvertureQualifications = model.addConstr((a == b), name='CouvertureQualifications')



Fonctions objectif

In [ ]:
## bénéfice total


obj = deliveryPointsPos + deliveryPointsNeg+ spiritMarketPos + spiritMarketNeg + oilMarket1Pos + oilMarket1Neg + oilMarket2Pos + oilMarket2Neg + oilMarket3Pos + oilMarket3Neg + retailerAPos + retailerANeg + retailerBPos + retailerBNeg 

model.setObjective(obj)

Model optimisation

In [ ]:
# Verify model formulation

# model.write('marketSharing.lp')

# Run optimization engine

model.optimize()